# Coffee Farm Condition Monitoring

This notebook provides code to analyze data collected using sensors deployed at the Dedan Kimathi University of Technology Coffee farm and display it for visual inspection.

We have deployed the following sensors
1. Ambient Temperature
1. Relative Humidity
1. Soil Temperature 
1. Soil Moisture

The aim is to use these data to help plan farming activities such as irrigation and fungicide application.

The sensors are connected to a [Nucleo F446re](https://os.mbed.com/platforms/ST-Nucleo-F446RE/) development board running code contained in this [repo](https://github.com/ciiram/nyeri-coffee).

In [ ]:
%matplotlib inline
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from datetime import datetime, timedelta

## Plots of Ambient and Soil Temperature in The Initial Deployment

In [ ]:
#load the data
df = pd.read_csv('data/2018-09-30-Data.csv')

# convert data type of time to datetime
df[['_source.time']] = df[['_source.time']].apply(pd.to_datetime)

# we are interested in time, ambient temperature and humidity, and soil temperature and humidity
df_coffee_farm = df[['_source.time','_source.dev_id','_source.temperature_2', '_source.relative_humidity_3', '_source.analog_in_4', '_source.analog_in_5']]
df_coffee_farm.columns = ['time','dev_id','Ambient Temperature', 'Relative Humidity', 'Soil Temperature', 'Soil Moisture']

# sorting based on time
df_coffee_farm.sort_values(by='time');


In [ ]:
for dev_name, gdf in df_coffee_farm.groupby('dev_id'):
        plt.figure()
        plt.title(dev_name)
        plt.plot(gdf.time + timedelta(hours=3), gdf['Ambient Temperature'],'bo')
        plt.plot(gdf.time + timedelta(hours=3), gdf['Soil Temperature'],'go')
        plt.legend(['Ambient Temperature', 'Soil Temperature']);
        plt.xticks(rotation=45);
        plt.ylim([0, 50]);
        plt.xlabel('Time');
        plt.ylabel('Temperature (Celcius)');
        plt.tight_layout()

## Second Deployment

In [ ]:
#load the data
ambient_temperature_df = pd.read_csv('data/ambient_temperature.csv', sep=";")
soil_temperature_df = pd.read_csv('data/soil_temperature.csv', sep=";")
soil_moisture_df = pd.read_csv('data/soil_moisture.csv', sep=";")

# convert data type of time to datetime and value to numeric
data_frames = [ambient_temperature_df, soil_temperature_df, soil_moisture_df]

for df in data_frames:
    df[['Time']] = df[['Time']].apply(pd.to_datetime)
    df[['Value']] = pd.to_numeric(df['Value'], errors='coerce')

plt.figure()
plt.plot(ambient_temperature_df[['Time']], ambient_temperature_df[['Value']], 'bo');
plt.plot(soil_temperature_df[['Time']], soil_temperature_df[['Value']], 'go');
plt.xticks(rotation=45);
plt.ylim([5, 30]);
plt.xlabel('Time');
plt.ylabel('Temperature (Celcius)');

plt.figure()
plt.plot(soil_moisture_df[['Time']], soil_moisture_df[['Value']], 'bo');
plt.xticks(rotation=45);
plt.ylim([-5, 100]);
plt.xlabel('Time');
plt.ylabel('Soil Moisture (%)');


## Modelling

We will experiment with curve fitting using the data we have collected. We will fit Gaussian process models with various kernels and assess the fits.

In [ ]:
import GPy
import numpy as np
from IPython.display import display

#prepare the data
temperature = ambient_temperature_df.values[:,2]
time_day = np.arange(len(temperature)) / 24 # normalize to days

time_day = time_day[~pd.isnull(temperature)]
temperature = temperature[~pd.isnull(temperature)]


#RBF Kernel
kernel = GPy.kern.RBF(input_dim=1, variance=1., lengthscale=1.)
m = GPy.models.GPRegression(time_day[:, None],temperature[:, None],kernel)
m.optimize_restarts(num_restarts = 10)
display(m)


In [ ]:
display(m)
fig = m.plot(plot_density=True)
GPy.plotting.show(fig, filename='rbf_fit');

We now try a periodic kernel on the same data.

In [ ]:
kernel = GPy.kern.PeriodicMatern32(input_dim=1, variance=1., lengthscale=1., period=1.)
m = GPy.models.GPRegression(time_day[:, None],temperature[:, None],kernel)
m.optimize_restarts(num_restarts = 10)

In [ ]:
display(m)
fig = m.plot(plot_density=True)
GPy.plotting.show(fig, filename='periodic_kernel');

We need to add bias.



In [ ]:
GPy.kern.Bias?

In [ ]:
kern2 = GPy.kern.Bias(1)

m = GPy.models.GPRegression(time_day[:, None],temperature[:, None],kernel + kern2)
m.optimize_restarts(num_restarts = 20)

display(m)
fig = m.plot(plot_density=True)
GPy.plotting.show(fig, filename='periodic_with_bias');